In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from numpy import ma

import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from pykalman import KalmanFilter
from subtracking import Grouse, calc_subspace_proj_error
from sklearn import decomposition

from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from ssidid.utility import get_subpop_stats, gen_data, draw_data, gen_pars
from ssidid.icml_scripts import run_default


lag_range = np.arange(0,20)
kl_ = np.max(lag_range) + 1
snr = (18., 18.)
p,n,T = 50, 10, 10000 + kl_
mmap, verbose = False, False

# settings for GROUSE
a_grouse = 1
max_epoch_size = 1000
max_iter_grouse = 200


idx_a, idx_b = np.arange(p), np.arange(p)
obs_scheme = ObservationScheme(p=p, T=T)
W = obs_scheme.comp_coocurrence_weights(lag_range, sso=True, idx_a=idx_a, idx_b=idx_b)

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e0/'

numExps = 20

for iExp in range(0,numExps):
    
    """
    # 2D 
    Q11 = 1e-2
    Q22 = 1e-0
    Q12, Q21 = 0.99*np.sqrt(Q11)*np.sqrt(Q22), 0.99*np.sqrt(Q11)*np.sqrt(Q22)    

    pars_true = {
        'C' : np.random.normal(size=(p,n)) / np.sqrt(n), 
        'mu0' : np.zeros(n),
        'A' : np.array([[0.95,  0.0],
                        [0.0,  0.0]]),
        'Q' : np.array([[Q11,Q12],[Q21,Q22]]),
        'R' : np.mean(snr) * np.ones(p)
    }
    
    # 3D, oscillating
    Q11 = 2e-3
    Q22 = 2e-3
    Q33 = 1e-1
    th = np.pi/30
    pars_true = {
        'C' : np.random.normal(size=(p,n)) / np.sqrt(n), 
        'A' : np.array([[ 0.99 * np.cos(th),  0.99 * np.sin(th), 0. ], 
                        [-0.99 * np.sin(th),  0.99 * np.cos(th), 0. ], 
                        [0.,                  0.,                0.95]]), 
        'Q' : np.outer(np.array([Q11, Q22, Q33]), np.array([Q11, Q22, Q33])) + 1e-15*np.eye(n),
        'R' : np.mean(snr) * np.ones(p),
        'mu0': np.zeros(n)
    }     
    """
    
    # 10
    n_gen = n
    numSlow = 9
    nr = numSlow

    Q11 = 2e-2
    Q22 = 1e-0
    CQ,dA = np.ones(n_gen), np.ones(n_gen)
    CQ[:numSlow], CQ[numSlow:] = Q11, Q22
    dA[:numSlow], dA[numSlow:] = np.linspace(0.99, 0.99, numSlow), np.linspace(0.0, 0.0, n_gen-numSlow)

    ev_c = np.exp(2 * 1j * np.pi * np.random.vonmises(mu=0, kappa=1000, size=(numSlow-nr)//2))
    ev_c = np.linspace(0.95, 0.99, (numSlow - nr)//2) * ev_c    
    ev_r = np.linspace(0.95, 0.99, nr)
    pars = gen_pars(p=0,n=numSlow, nr=nr, ev_c = ev_c, ev_r = ev_r, snr = (0.,0.))
    A = pars['A'].copy()
    del pars
    
    pars_true = {
        'C' : np.random.normal(size=(p,n_gen)) / np.sqrt(n_gen), 
        'mu0' : np.zeros(n_gen),
        #'A' : np.diag(dA),        
        'A' : np.zeros((n,n)),
        'R' : np.mean(snr) * np.ones(p),
        'Q' : np.diag(CQ)
    }            
    #pars_true['Q'] = 0.9 * np.diag(np.hstack((CQ[:numSlow], np.zeros(n_gen-numSlow)) )) \
    #               + 1e-12*np.eye(n_gen)                                             
    #idx = np.arange(numSlow+1)
    #pars_true['Q'][np.ix_(idx,idx)] += 0.1 * np.outer(np.sqrt(CQ[idx]), np.sqrt(CQ[idx]))
    #idx = np.hstack((np.arange(numSlow), n_gen-1))
    #pars_true['Q'][np.ix_(idx,idx)] += 0.3 * np.outer(np.sqrt(CQ[idx]), np.sqrt(CQ[idx]))
    #                               
    #pars_true['A'][:numSlow, numSlow] = np.array([[ 0.99 * np.cos(th),  0.99 * np.sin(th)], 
    #                                              [-0.99 * np.sin(th),  0.99 * np.cos(th)]])
    pars_true['A'][:numSlow, :numSlow] = A
    
    pars_true['C'] = la.orth(pars_true['C']) * np.sqrt(p)
    #pars_true['C'] /= np.atleast_1d(np.linalg.norm(pars_true['C'],axis=0))
    pars_true['Pi'] = np.atleast_2d(sp.linalg.solve_discrete_lyapunov(pars_true['A'], pars_true['Q']))
    pars_true['V0'] = pars_true['Pi'].copy()
    
    print('eigvals A', np.linalg.eigvals(pars_true['A']))

    rnd_seed = np.mod(int(time.time() * 10000), 10000)
    np.random.seed(rnd_seed)
    x, _ = draw_data(pars_true, 2*T)
    x = x[T:,:]
    
    x = (x - x.mean(axis=0))/np.std(x,axis=0)
    x[:,:numSlow] /= np.sqrt(numSlow)
    #x /= np.sum(np.var(x, axis=0))
    
    eps = np.sqrt(pars_true['R']).reshape(1,p) * np.random.normal(size=(x.shape[0],p))
    y = x.dot(pars_true['C'].T) + eps
    y -= y.mean(axis=0)

    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.imshow(pars_true['C'], interpolation='None')
    plt.colorbar()
    plt.subplot(2,2,2)
    plt.plot(x[:,0], x[:,1])
    plot_range = np.arange(T)
    plt.subplot(2,2,4)
    plt.plot(x[plot_range,:])
    plt.show()


    # setting up observation scheme
    Qs, Om = f_l2_Hankel_comp_Q_Om(n=n,y=y,lag_range=lag_range,obs_scheme=obs_scheme,
                          idx_a=idx_a,idx_b=idx_b,W=W,sso=True,
                          mmap=mmap,data_path=data_path,ts=None,ms=None)   
    print('var[x]', np.var(x, axis=0))

    plt.figure(figsize=(20,3))
    plt.imshow((np.hstack([Qs[m] for m in range(len(lag_range))])), interpolation='None')
    plt.show()


    pars_true['X'] = np.vstack([np.linalg.matrix_power(pars_true['A'],m).dot(pars_true['Pi']) for m in lag_range])
    print('true param. loss: ', f_l2_Hankel_nl(C=pars_true['C'],X=pars_true['X'],R=pars_true['R'],
                               Qs=Qs,Om=Om,lag_range=lag_range,ms=range(len(lag_range)),idx_a=idx_a,idx_b=idx_b))        

    print_slim(Qs,Om,lag_range,pars_true,idx_a,idx_b,None,False,data_path)

    # do PCA
    pca = decomposition.PCA(n_components=n)
    pca.fit(y[:,idx_a])
    C_pca =pca.components_.T
    pars_pca = { 'C' : pca.components_.T.copy(), 
                 'X' : np.vstack([np.diag(pca.explained_variance_) for m in range(len(lag_range)) ]),
                 'R' : np.zeros(p)}
    Qpca = pars_pca['C'].dot(pars_pca['X'][:n,:]).dot(pars_pca['C'].T)
    plt.imshow(Qpca, interpolation='None')
    plt.show()
    print_slim(Qs,Om,lag_range,pars_pca,np.arange(len(idx_a),dtype=int),np.arange(len(idx_a),dtype=int),
               None,False,data_path)
    print('inst. corrcoeff', np.corrcoef(Qpca.reshape(-1), Qs[0].reshape(-1))[0,1])
    print('prin. angles',  principal_angle(pars_true['C'], pars_pca['C']) )    

    rnd_seed = np.random.get_state()
    pars_est={
        'C' : np.random.normal(size=(p,n)) / np.sqrt(n),
        'A' : 0 * np.diag(np.linspace(0.9, 0.95, n)),
        'B' : np.eye(n),
        'Pi': np.eye(n),
        'R' : np.zeros(p)
    }
    pars_est['X'] = np.vstack([ np.linalg.matrix_power(pars_est['A'],m).dot(pars_est['Pi']) for m in lag_range])    
    pars_est, traces, ts= run_default(
                alphas    = (0.025, 0.01), 
                b1s       = (0.99, 0.95), 
                a_decays  = (0.98, 0.98), 
                batch_sizes = (1, 10), 
                max_zip_sizes =  (1000,1000), 
                max_iters = (100, 100),
                parametrizations = ('nl', 'nl'),
                pars_est=pars_est, pars_true=pars_true, n=n, 
                y=y, sso=True, obs_scheme=obs_scheme, lag_range=lag_range, 
                idx_a=idx_a, idx_b=idx_b,Qs=Qs,Om=Om, W=W,
                traces=[[], [], []], ts = [])        
    print('prin. angles',  principal_angle(pars_true['C'], pars_est['C']) )    
    print('slow and fast subspaces')
    print(principal_angle(pars_true['C'][:,:numSlow], pars_est['C']),principal_angle(pars_true['C'][:,numSlow:], pars_est['C']))

    
    # fit GROUSE
    traces_g = np.zeros((max_iter_grouse, n+1))
    pars_est_g = 'default'
    tracker = Grouse(p, n, a_grouse )
    get_obs = obs_scheme.gen_get_observed()
    print('\n - GROUSE')
    tracker.step = a_grouse
    ct = 1.
    t_g = time.time()
    get_obs = obs_scheme.gen_get_observed()

    for i in range(max_iter_grouse):
        if np.mod(i,max_iter_grouse//10) == 0:
            print('finished % ' + str((100*i)//max_iter_grouse))
        idx = np.random.permutation(T-np.max(lag_range)-1)
        idx = idx[:max_epoch_size] if len(idx) > max_epoch_size else idx
        for j in range(len(idx)):
            obs_idx =  np.zeros((p,1), dtype=bool)
            obs_idx[get_obs(idx[j])] = True
            tracker.consume(y[idx[j],:].reshape(-1,1), obs_idx)
            ct += 1     
            tracker.step = a_grouse / ct

        traces_g[i] = np.hstack((calc_subspace_proj_error(pars_true['C'], tracker.U), principal_angle(pars_true['C'], tracker.U)))
    t_g = time.time() - t_g
    pars_est_g = {'C' : tracker.U.copy()}
    print('prin. angles',  principal_angle(pars_true['C'], pars_est_g['C']) )        
    print('slow and fast subspaces')
    print(principal_angle(pars_true['C'][:,:numSlow], pars_est_g['C']),principal_angle(pars_true['C'][:,numSlow:], pars_est_g['C']))
    
    print('filtering data') 
    obs_scheme.gen_mask_from_scheme()
    tracker = Grouse(p, n, 0. )
    tracker.U = pars_est_g['C'].copy()
    x_g = np.zeros((T,n))
    for t in range (T):
        x_g[t,:] = tracker._project(y[t,:].reshape(p,1), obs_scheme.mask[t,:].reshape(p,1)).reshape(-1)
    obs_scheme.mask = None    
    obs_scheme.use_mask = False
    lag_range_g = lag_range.copy()
    kl_ = np.max(lag_range_g) + 1
    print('extracting dynamics parameters') 
    pars_est_g['X'] = np.vstack([np.cov(x_g[m:-(kl_+1)+m, :].T, x_g[:-(kl_+1), :].T)[:n,n:] for m in lag_range_g])
    pars_est_g['A'] = np.linalg.lstsq(pars_est_g['X'][:(len(lag_range_g)-1)*n,:], pars_est_g['X'][n:len(lag_range_g)*n,:])[0]
    pars_est_g['Pi'] = (pars_est_g['X'][:n,:] + pars_est_g['X'][:n,:].T)/2 
    ev_est = np.linalg.eigvals(pars_est_g['A'])
    del x_g        
    pars_est_g['R'] = np.zeros(p)
    print('GROUSE param. loss: ', f_l2_Hankel_nl(C=pars_est_g['C'],X=pars_est_g['X'],R=pars_est_g['R'],
                                   Qs=Qs,Om=Om,lag_range=lag_range,ms=range(len(lag_range)),idx_a=idx_a,idx_b=idx_b))        

    print_slim(Qs,Om,lag_range,pars_est_g,idx_a,idx_b,None,False,data_path)
    


    plt.subplot(1,2,1)
    plt.plot(traces_g[:,1:])
    plt.title('subspace proj. error (GROUSE)')
    plt.subplot(1,2,2)
    plt.loglog(traces_g[:,1:])
    plt.title('subspace proj. error (GROUSE)')
    plt.show()
    
    save_dict = {'p' : p, 'n' : n, 'T' : T, 'snr' : snr,
                 'obs_scheme' : obs_scheme,
                 'lag_range' : lag_range,
                 'pars_true' : pars_true,
                 'pars_est' : pars_est,
                 'traces' : traces, 
                 'pars_est_g' : pars_est_g,
                 'traces_g' : traces_g, 
                 'pars_est_pca' : pars_pca,
                 'idx_a' : idx_a,
                 'idx_b' : idx_a,
                 't' : t,
                 't_g' : t_g,
                 'W' : W,
                 'Qs' : Qs,
                 'Om' : Om,
                 'rnd_seed' : rnd_seed}

    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T) + '_runExp' + str(iExp)
    np.savez(data_path + file_name, save_dict)    
    


In [ ]:
# settings for GROUSE
a_grouse = .1
max_epoch_size = 1000
max_iter_grouse = 500

traces_g = np.zeros((max_iter_grouse, n+1))
pars_est_g = 'default'
#tracker = Grouse(p, n, a_grouse )
get_obs = obs_scheme.gen_get_observed()

# fit GROUSE
print('\n - GROUSE')
tracker.step = a_grouse
ct = 1.
t_g = time.time()
get_obs = obs_scheme.gen_get_observed()

for i in range(max_iter_grouse):
    if np.mod(i,max_iter_grouse//10) == 0:
        print('finished % ' + str((100*i)//max_iter_grouse))
    idx = np.random.permutation(T-np.max(lag_range)-1)
    idx = idx[:max_epoch_size] if len(idx) > max_epoch_size else idx
    for j in range(len(idx)):
        obs_idx =  np.zeros((p,1), dtype=bool)
        obs_idx[get_obs(idx[j])] = True
        tracker.consume(y[idx[j],:].reshape(-1,1), obs_idx)
        ct += 1     
        tracker.step = a_grouse / ct

    traces_g[i] = np.hstack((calc_subspace_proj_error(pars_true['C'], tracker.U), principal_angle(pars_true['C'], tracker.U)))
t_g = time.time() - t_g
pars_est_g = {'C' : tracker.U.copy()}


plt.subplot(1,2,1)
plt.plot(traces_g[:,1:])
plt.title('subspace proj. error (GROUSE)')
plt.subplot(1,2,2)
plt.loglog(traces_g[:,1:])
plt.title('subspace proj. error (GROUSE)')
plt.show()

print('filtering data') 
obs_scheme.gen_mask_from_scheme()
tracker = Grouse(p, n, 0. )
tracker.U = pars_est_g['C'].copy()
x_g = np.zeros((T,n))
for t in range (T):
    x_g[t,:] = tracker._project(y[t,:].reshape(p,1), obs_scheme.mask[t,:].reshape(p,1)).reshape(-1)
obs_scheme.mask = None    
obs_scheme.use_mask = False

lag_range_g = lag_range.copy()
kl_ = np.max(lag_range_g) + 1
print('extracting dynamics parameters') 
pars_est_g['X'] = np.vstack([np.cov(x_g[m:-(kl_+1)+m, :].T, x_g[:-(kl_+1), :].T)[:n,n:] for m in lag_range_g])
pars_est_g['A'] = np.linalg.lstsq(pars_est_g['X'][:(len(lag_range_g)-1)*n,:], pars_est_g['X'][n:len(lag_range_g)*n,:])[0]
pars_est_g['Pi'] = (pars_est_g['X'][:n,:] + pars_est_g['X'][:n,:].T)/2 
ev_est = np.linalg.eigvals(pars_est_g['A'])
del x_g        

pars_est_g['R'] = np.zeros(p)
print_slim(Qs,Om,lag_range,pars_est_g,idx_a,idx_b,None,False,data_path)

In [ ]:
principal_angle(pars_true['C'][:,0], pars_est_g['C']), principal_angle(pars_true['C'][:,1], pars_est_g['C'])

In [ ]:
principal_angle(pars_true['C'][:,0], pars_est['C']), principal_angle(pars_true['C'][:,1], pars_est['C'])

In [ ]:
    n = 4


    numSlow = 1
    Q11 = 1e-3
    Q22 = 1e-1
    #Q12, Q21 = 0.9999*np.sqrt(Q11)*np.sqrt(Q22), 0.9999*np.sqrt(Q11)*np.sqrt(Q22)    
    CQ,dA = np.ones(n), np.ones(n)
    CQ[:numSlow], CQ[numSlow:] = Q11, Q22
    dA[:numSlow], dA[numSlow:] = np.linspace(0.99, 0.99, numSlow), np.linspace(0.0, 0.0, n-numSlow)

    pars_true = {
        'C' : np.random.normal(size=(p,n)) / np.sqrt(n), 
        'mu0' : np.zeros(n),
        'A' : np.diag(dA),
        'Q' : np.diag(CQ), #np.outer(CQ, CQ) + 1e-12*np.eye(n),
        'R' : np.mean(snr) * np.ones(p)
    }            
    pars_true['C'] = la.orth(pars_true['C']) * np.sqrt(p)
    pars_true['Pi'] = np.atleast_2d(sp.linalg.solve_discrete_lyapunov(pars_true['A'], pars_true['Q']))
    pars_true['V0'] = pars_true['Pi'].copy()

     
    
    
    pars_true['C'] = la.orth(pars_true['C']) * np.sqrt(p)
    pars_true['Pi'] = np.atleast_2d(sp.linalg.solve_discrete_lyapunov(pars_true['A'], pars_true['Q']))
    pars_true['V0'] = pars_true['Pi'].copy()
    pars_true['X'] = np.vstack([np.linalg.matrix_power(pars_true['A'],m).dot(pars_true['Pi']) for m in lag_range])


    rnd_seed = np.mod(int(time.time() * 10000), 10000)
    np.random.seed(rnd_seed)
    x, _ = draw_data(pars_true, 2*T)
    x = x[T:,:]

    x = (x - x.mean(axis=0))/np.std(x,axis=0)
    x[:,0] /= np.sqrt(10)


    plt.figure(figsize=(20,3))
    plt.imshow(pars_true['X'].T, interpolation='None')
    plt.show()

    plt.plot(pars_true['X'])
    plt.show()
    np.corrcoef(x.T), pars_true['Pi']


# plotting

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from numpy import ma

import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from pykalman import KalmanFilter
from subtracking import Grouse, calc_subspace_proj_error
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from ssidid.utility import get_subpop_stats, gen_data, draw_data
from ssidid.icml_scripts import run_default


color_us =            (244/255, 152/255, 25/255)
color_us_bitflipped = 'k'  #(0,0,0)
color_GROUSE =        'm'  #0.7 * np.array((244/255, 152/255, 25/255))

lag_range = np.arange(20)
kl_ = np.max(lag_range) + 1
snr = (10., 10.)
p,n,T = 30, 2, 1000 + kl_
mmap, verbose = False, False

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e0/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'
numExps = 20


plt.figure(figsize=(20,5))

prin_angl_final = np.zeros((numExps,2,n))
for iExp in range(numExps):


    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T) + '_runExp' + str(iExp)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()    
    
    pars_true = load_file['pars_true']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    
    prin_angl_final[iExp,0,:] = principal_angle(pars_true['C'], 
                                              pars_est['C'])
    prin_angl_final[iExp,1,:] = principal_angle(pars_true['C'], 
                                              pars_est_g['C'])
    
    plt.subplot(1,4,1)
    plt.plot(np.arange(0, 2*len(load_file['traces_g'][:, 0]), 2),
             load_file['traces_g'][:, 0], color=color_GROUSE, linewidth=1.5)
    plt.plot(load_file['traces'][2][:,0], color=color_us, linewidth=1.5)
    plt.title('subspace projection errors')
    plt.xlabel('epoch')
    plt.ylabel('subsp. proj. err.')
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid'])
    #plt.plot(principal_angle[load_file['traces_g'][:, 1:]], 'm')

    plt.subplot(1,4,2)
    plt.plot(load_file['traces_g'][-1, 0], load_file['traces'][2][-1, 0], 'ko', linewidth=3)
    plt.xlabel('subsp. proj. err. GROUSE')
    plt.ylabel('subsp. proj. err. ssidid')
    plt.plot([0.1,1], [0.1,1], color='k')
    plt.box('off')
    plt.title('final subsp. proj. err, GROUSE vs. us')
    
    
    plt.subplot(1,4,3)
    plt.plot(load_file['traces_g'][-1, 1:], 'o', color=color_GROUSE, linewidth=3)
    plt.plot(np.arange(n)+0.05, load_file['traces'][2][-1, 1:], 'o', color=color_us, linewidth=3)
    plt.axis([-0.1, 1.1, 0, 0.9])
    plt.title('final prinicpal angles betw. est. and true C')
    plt.xlabel('latent mode')
    plt.ylabel('principal angle')
    plt.xticks([0, 1], ['#1', '#2'])
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid'], loc=2)
    
    plt.subplot(1,4,4)
    plt.plot(load_file['traces_g'][-1, 1], load_file['traces'][2][-1, 1], 'ko', linewidth=3)
    plt.plot(load_file['traces_g'][-1, 2], load_file['traces'][2][-1, 2], 'kx', linewidth=3)
    plt.xlabel('principal angle GROUSE')
    plt.ylabel('principal angle ssidid')
    plt.legend(['mode #1', 'mode #2'], loc =2)
    plt.plot([0,1], [0,1], color='k')
    plt.box('off')
    plt.title('final principal angles, GROUSE vs. us')
    
    plt.savefig(fig_path + 'illustration_dynamics_dominated_p30_hard.pdf')
        
plt.show()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from numpy import ma

import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from pykalman import KalmanFilter
from subtracking import Grouse, calc_subspace_proj_error
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from ssidid.utility import get_subpop_stats, gen_data, draw_data
from ssidid.icml_scripts import run_default


color_us =            (244/255, 152/255, 25/255)
color_us_bitflipped = 'k'  #(0,0,0)
color_GROUSE =        'm'  #0.7 * np.array((244/255, 152/255, 25/255))

lag_range = np.arange(10)
kl_ = np.max(lag_range) + 1
snr = (10., 10.)
p,n,T = 30, 2, 3000 + kl_
mmap, verbose = False, False

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e0/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'
numExps = 20


plt.figure(figsize=(20,5))

prin_angl_final = np.zeros((numExps,2,n))
for iExp in range(numExps):


    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T) + '_runExp' + str(iExp)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()    
    
    pars_true = load_file['pars_true']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    
    prin_angl_final[iExp,0,:] = principal_angle(pars_true['C'], 
                                              pars_est['C'])
    prin_angl_final[iExp,1,:] = principal_angle(pars_true['C'], 
                                              pars_est_g['C'])
    
    plt.subplot(1,4,1)
    plt.plot(np.arange(0, 2*len(load_file['traces_g'][:, 0]), 2),
             load_file['traces_g'][:, 0], color=color_GROUSE, linewidth=1.5)
    plt.plot(load_file['traces'][2][:,0], color=color_us, linewidth=1.5)
    plt.title('subspace projection errors')
    plt.xlabel('epoch')
    plt.ylabel('subsp. proj. err.')
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid'])
    #plt.plot(principal_angle[load_file['traces_g'][:, 1:]], 'm')

    plt.subplot(1,4,2)
    plt.plot(load_file['traces_g'][-1, 0], load_file['traces'][2][-1, 0], 'ko', linewidth=3)
    plt.xlabel('subsp. proj. err. GROUSE')
    plt.ylabel('subsp. proj. err. ssidid')
    plt.plot([0.1,1], [0.1,1], color='k')
    plt.box('off')
    plt.title('final subsp. proj. err, GROUSE vs. us')
    
    
    plt.subplot(1,4,3)
    plt.plot(load_file['traces_g'][-1, 1:], 'o', color=color_GROUSE, linewidth=3)
    plt.plot(np.arange(n)+0.05, load_file['traces'][2][-1, 1:], 'o', color=color_us, linewidth=3)
    plt.axis([-0.1, 1.1, 0, 0.9])
    plt.title('final prinicpal angles betw. est. and true C')
    plt.xlabel('latent mode')
    plt.ylabel('principal angle')
    plt.xticks([0, 1], ['#1', '#2'])
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid'], loc=2)
    
    plt.subplot(1,4,4)
    plt.plot(load_file['traces_g'][-1, 1], load_file['traces'][2][-1, 1], 'ko', linewidth=3)
    plt.plot(load_file['traces_g'][-1, 2], load_file['traces'][2][-1, 2], 'kx', linewidth=3)
    plt.xlabel('principal angle GROUSE')
    plt.ylabel('principal angle ssidid')
    plt.legend(['mode #1', 'mode #2'], loc =2)
    plt.plot([0,1], [0,1], color='k')
    plt.box('off')
    plt.title('final principal angles, GROUSE vs. us')
    
    #plt.savefig(fig_path + 'illustration_dynamics_dominated_p30_easiyer.pdf')
        
plt.show()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from numpy import ma

import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from pykalman import KalmanFilter
from subtracking import Grouse, calc_subspace_proj_error
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from ssidid.utility import get_subpop_stats, gen_data, draw_data
from ssidid.icml_scripts import run_default


color_us =            (244/255, 152/255, 25/255)
color_us_bitflipped = 'k'  #(0,0,0)
color_GROUSE =        'm'  #0.7 * np.array((244/255, 152/255, 25/255))

lag_range = np.arange(10)
kl_ = np.max(lag_range) + 1
snr = (10., 10.)
p,n,T = 10, 2, 5000 + kl_
mmap, verbose = False, False

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

def comp_slim(Qs,Om,lag_range,pars,idx_a,idx_b,traces=None,mmap=False,data_path=None):

    kl = len(lag_range)
    
    out = np.zeros(len(lag_range))
    
    p,n = pars['C'].shape
    pa, pb = idx_a.size, idx_b.size
    idx_ab = np.intersect1d(idx_a, idx_b)
    idx_a_ab = np.where(np.in1d(idx_a, idx_ab))[0]
    idx_b_ab = np.where(np.in1d(idx_b, idx_ab))[0]
    for m in range(kl): 
        m_ = lag_range[m] 
        Qrec = pars['C'][idx_a,:].dot(pars['X'][m*n:(m+1)*n, :]).dot(pars['C'][idx_b,:].T) 
        if m_ == 0:
            Qrec[np.ix_(idx_a_ab, idx_b_ab)] += np.diag(pars['R'][idx_ab])
        if mmap:
            Q = np.memmap(data_path+'Qs_'+str(m_), dtype=np.float, mode='r', shape=(pa,pb))
        else:
            Q = Qs[m]
        out[m] = np.corrcoef( Qrec[Om[m]].reshape(-1), (Qs[m][Om[m]]).reshape(-1) )[0,1]
        if mmap:
            del Q
    return out

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e0/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'
numExps = 20


plt.figure(figsize=(20,5))

prin_angl_final = np.zeros((numExps,2,n))
for iExp in range(numExps):


    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T) + '_runExp' + str(iExp)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()    
    
    pars_true = load_file['pars_true']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    
    prin_angl_final[iExp,0,:] = principal_angle(pars_true['C'], 
                                              pars_est['C'])
    prin_angl_final[iExp,1,:] = principal_angle(pars_true['C'], 
                                              pars_est_g['C'])
    
    plt.subplot(1,4,1)
    plt.plot(np.arange(0, 2*len(load_file['traces_g'][:, 0]), 2),
             load_file['traces_g'][:, 0], color=color_GROUSE, linewidth=1.5)
    plt.plot(load_file['traces'][2][:,0], color=color_us, linewidth=1.5)
    plt.title('subspace projection errors')
    plt.xlabel('epoch')
    plt.ylabel('subsp. proj. err.')
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid'])
    #plt.plot(principal_angle[load_file['traces_g'][:, 1:]], 'm')

    plt.subplot(1,4,2)
    plt.plot(load_file['traces_g'][-1, 0], load_file['traces'][2][-1, 0], 'ko', linewidth=3)
    plt.xlabel('subsp. proj. err. GROUSE')
    plt.ylabel('subsp. proj. err. ssidid')
    plt.plot([0.1,1], [0.1,1], color='k')
    plt.box('off')
    plt.title('final subsp. proj. err, GROUSE vs. us')
    
    
    plt.subplot(1,4,3)
    plt.plot(load_file['traces_g'][-1, 1:], 'o', color=color_GROUSE, linewidth=3)
    plt.plot(np.arange(n)+0.05, load_file['traces'][2][-1, 1:], 'o', color=color_us, linewidth=3)
    plt.axis([-0.1, 1.1, 0, 0.9])
    plt.title('final prinicpal angles betw. est. and true C')
    plt.xlabel('latent mode')
    plt.ylabel('principal angle')
    plt.xticks([0, 1], ['#1', '#2'])
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid'], loc=2)
    
    plt.subplot(1,4,4)
    plt.plot(load_file['traces_g'][-1, 1], load_file['traces'][2][-1, 1], 'ko', linewidth=3)
    plt.plot(load_file['traces_g'][-1, 2], load_file['traces'][2][-1, 2], 'kx', linewidth=3)
    plt.xlabel('principal angle GROUSE')
    plt.ylabel('principal angle ssidid')
    plt.legend(['mode #1', 'mode #2'], loc =2)
    plt.plot([0,1], [0,1], color='k')
    plt.box('off')
    plt.title('final principal angles, GROUSE vs. us')
    
    #plt.savefig(fig_path + 'illustration_dynamics_dominated.pdf')
        
plt.show()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from numpy import ma

import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from pykalman import KalmanFilter
from subtracking import Grouse, calc_subspace_proj_error
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from ssidid.utility import get_subpop_stats, gen_data, draw_data
from ssidid.icml_scripts import run_default


color_us =            (244/255, 152/255, 25/255)
color_us_bitflipped = 'k'  #(0,0,0)
color_GROUSE =        'm'  #0.7 * np.array((244/255, 152/255, 25/255))

lag_range = np.arange(0,21)
kl_ = np.max(lag_range) + 1
kl_ = np.max(lag_range) + 1
snr = (0.5, 0.5)
p,n,T = 10, 4, 10000 + kl_

def comp_slim(Qs,Om,lag_range,pars,idx_a,idx_b,traces=None,mmap=False,data_path=None):

    kl = len(lag_range)
    
    out = np.zeros(len(lag_range))
    
    p,n = pars['C'].shape
    pa, pb = idx_a.size, idx_b.size
    idx_ab = np.intersect1d(idx_a, idx_b)
    idx_a_ab = np.where(np.in1d(idx_a, idx_ab))[0]
    idx_b_ab = np.where(np.in1d(idx_b, idx_ab))[0]
    for m in range(kl): 
        m_ = lag_range[m] 
        Qrec = pars['C'][idx_a,:].dot(pars['X'][m*n:(m+1)*n, :]).dot(pars['C'][idx_b,:].T) 
        if m_ == 0:
            Qrec[np.ix_(idx_a_ab, idx_b_ab)] += np.diag(pars['R'][idx_ab])
        if mmap:
            Q = np.memmap(data_path+'Qs_'+str(m_), dtype=np.float, mode='r', shape=(pa,pb))
        else:
            Q = Qs[m]
        out[m] = np.corrcoef( Qrec[Om[m]].reshape(-1), (Qs[m][Om[m]]).reshape(-1) )[0,1]
        if mmap:
            del Q
    return out

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e0/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'
numExps = 10


plt.figure(figsize=(15,8))

prin_angl_final = np.zeros((numExps,3,n+1))
for iExp in range(numExps):


    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T) + '_runExp' + str(iExp)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()    
    
    pars_true = load_file['pars_true']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    pars_est_pca = load_file['pars_est_pca']
    Qs = load_file['Qs']
    Om = load_file['Om']
    
    prin_angl_final[iExp,0,0] = principal_angle(pars_true['C'][:,0], 
                                              pars_est['C'])
    prin_angl_final[iExp,1,0] = principal_angle(pars_true['C'][:,0], 
                                              pars_est_g['C'])
    prin_angl_final[iExp,2,0] = principal_angle(pars_true['C'][:,0], 
                                              pars_est_pca['C'])

    prin_angl_final[iExp,0,1:] = principal_angle(pars_true['C'][:,1:], 
                                              pars_est['C'])
    prin_angl_final[iExp,1,1:] = principal_angle(pars_true['C'][:,1:], 
                                              pars_est_g['C'])
    prin_angl_final[iExp,2,1:] = principal_angle(pars_true['C'][:,1:], 
                                              pars_est_pca['C'])
    
    
    temp_corrs_t = comp_slim(Qs,Om,lag_range,pars_true,idx_a,idx_b)
    temp_corrs   = comp_slim(Qs,Om,lag_range,pars_est,idx_a,idx_b)
    temp_corrs_g = comp_slim(Qs,Om,lag_range,pars_est_g,idx_a,idx_b)
    
    plt.subplot(1,2,1)
    plt.plot(lag_range[0:], temp_corrs_g[0:], 'o-', color=color_GROUSE)
    plt.plot(lag_range[0:], temp_corrs[0:], 'o-', color=color_us)
    plt.plot(lag_range[0:], temp_corrs_t[0:], 'ko-')
    plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on", tickdir='out')    
    plt.box('off')
    plt.axis([-.1, np.max(lag_range)+0.1, 0, 1])
    plt.xlabel('time-lag')
    plt.ylabel('corr. of cov.')
    plt.title('time-lagged correlations')
    plt.legend(['GROUSE', 'ssidid', 'true params.'], loc=5)
    plt.subplot(1,2,2)
    plt.plot(np.arange(n), prin_angl_final[iExp,1,1:], 'o', color=color_GROUSE, linewidth=3)
    plt.plot(np.arange(n)-0.05, prin_angl_final[iExp,0,1:], 'o', color=color_us, linewidth=3)
    plt.plot(np.arange(n)+0.05, prin_angl_final[iExp,2,1:], 'o', color='b', linewidth=3)
    plt.axis([-0.1, n+.1, 0, 1.1])
    plt.title('prin. angles betw. est. and true C')
    plt.xlabel('latent mode')
    plt.ylabel('principal angle')
    plt.xticks([0, 1, 2, 3, 4], ['#1', '#2', '#3', '#4', '#5 (weak slow mode)'])
    plt.box('off')
    plt.legend(['GROUSE', 'ssidid', 'PCA'], loc=2)
    plt.plot(4, prin_angl_final[iExp,0,0], 'o', color=color_us, linewidth=3)
    plt.plot(4.05, prin_angl_final[iExp,1,0], 'o', color=color_GROUSE, linewidth=3)
    plt.plot(3.95, prin_angl_final[iExp,2,0], 'o', color='b', linewidth=3)
        
        
    plt.savefig(fig_path + 'illustration_dynamics_dominated_fewer_latents.pdf')
        
plt.show()

In [ ]:
% matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm

import numpy as np
from numpy import ma

import scipy as sp
from scipy import linalg as la
import glob, os, psutil, time

from pykalman import KalmanFilter
from subtracking import Grouse, calc_subspace_proj_error
from ssidid.SSID_Hankel_loss import run_bad, plot_slim, print_slim, f_l2_Hankel_nl, f_l2_Hankel_comp_Q_Om
from ssidid import ObservationScheme, progprint_xrange
from ssidid.utility import get_subpop_stats, gen_data, draw_data
from ssidid.icml_scripts import run_default


color_us =            (244/255, 152/255, 25/255)
color_us_bitflipped = 'k'  #(0,0,0)
color_GROUSE =        'm'  #0.7 * np.array((244/255, 152/255, 25/255))

lag_range = np.arange(0,21)
kl_ = np.max(lag_range) + 1
p,n,T = 50, 10, 10000 + kl_

#lag_range = np.arange(0,10)
#kl_ = np.max(lag_range) + 1
#p,n,T = 10, 4, 5000 + kl_


def comp_slim(Qs,Om,lag_range,pars,idx_a,idx_b,traces=None,mmap=False,data_path=None):

    kl = len(lag_range)
    
    out = np.zeros(len(lag_range))
    
    p,n = pars['C'].shape
    pa, pb = idx_a.size, idx_b.size
    idx_ab = np.intersect1d(idx_a, idx_b)
    idx_a_ab = np.where(np.in1d(idx_a, idx_ab))[0]
    idx_b_ab = np.where(np.in1d(idx_b, idx_ab))[0]
    for m in range(kl): 
        m_ = lag_range[m] 
        Qrec = pars['C'][idx_a,:].dot(pars['X'][m*n:(m+1)*n, :]).dot(pars['C'][idx_b,:].T) 
        if m_ == 0:
            Qrec[np.ix_(idx_a_ab, idx_b_ab)] += np.diag(pars['R'][idx_ab])
        if mmap:
            Q = np.memmap(data_path+'Qs_'+str(m_), dtype=np.float, mode='r', shape=(pa,pb))
        else:
            Q = Qs[m]
        out[m] = np.corrcoef( Qrec[Om[m]].reshape(-1), (Qs[m][Om[m]]).reshape(-1) )[0,1]
        if mmap:
            del Q
    return out

def principal_angle(A, B):
    "A and B must be column-orthogonal."    
    A = np.atleast_2d(A).T if (A.ndim<2) else A
    B = np.atleast_2d(B).T if (B.ndim<2) else B
    A = la.orth(A)
    B = la.orth(B)
    svd = la.svd(A.T.dot(B))
    return np.arccos(np.minimum(svd[1], 1.0)) / (np.pi/2)

data_path =  '/home/mackelab/Desktop/Projects/Stitching/results/icml_e0/'
fig_path =  '/home/mackelab/Desktop/Projects/Stitching/figures/'
numExps = 10

temp_corrs_t = np.zeros((len(lag_range), numExps))
temp_corrs   = np.zeros((len(lag_range), numExps))
temp_corrs_g = np.zeros((len(lag_range), numExps))

tmp_pca = np.zeros(numExps)
tmp   = np.zeros(numExps)
tmp_g = np.zeros(numExps)

plt.figure(figsize=(15,5))

prin_angl_final = np.zeros((numExps,3,n))
for iExp in range(numExps):


    file_name = 'p' + str(p) + 'n' + str(n) + 'T' + str(T) + '_runExp' + str(iExp)
    load_file = np.load(data_path + file_name + '.npz')['arr_0'].tolist()    
    
    pars_true = load_file['pars_true']
    pars_est = load_file['pars_est']
    pars_est_g = load_file['pars_est_g']
    pars_est_pca = load_file['pars_est_pca']
    Qs = load_file['Qs']
    Om = load_file['Om']
    idx_a, idx_b = load_file['idx_a'], load_file['idx_b']
    prin_angl_final[iExp,0,:] = principal_angle(pars_true['C'], 
                                              pars_est['C'])
    prin_angl_final[iExp,1,:] = principal_angle(pars_true['C'], 
                                              pars_est_g['C'])
    prin_angl_final[iExp,2,:] = principal_angle(pars_true['C'], 
                                              pars_est_pca['C'])

    #prin_angl_final[iExp,0,1:] = principal_angle(pars_true['C'][:,1:], 
    #                                          pars_est['C'])
    #prin_angl_final[iExp,1,1:] = principal_angle(pars_true['C'][:,1:], 
    #                                          pars_est_g['C'])
    #prin_angl_final[iExp,2,1:] = principal_angle(pars_true['C'][:,1:], 
    #                                          pars_est_pca['C'])
    
    
    temp_corrs_t[:,iExp] = comp_slim(Qs,Om,lag_range,pars_true,idx_a,idx_b)
    temp_corrs[:,iExp]   = comp_slim(Qs,Om,lag_range,pars_est,idx_a,idx_b)
    temp_corrs_g[:,iExp] = comp_slim(Qs,Om,lag_range,pars_est_g,idx_a,idx_b)
    
    plt.subplot(1,3,1)
    idxl = np.arange(1, len(lag_range))
    lags = lag_range[idxl]
    
    plt.plot(lags, temp_corrs_g[idxl,iExp], 'o-', color=color_GROUSE)
    plt.plot(lags, temp_corrs[idxl,iExp], 'o-', color=color_us)
    #plt.plot(lags, temp_corrs_t[idxl,iExp], 'ko-')
    plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on", tickdir='out')    
    plt.box('off')
    #plt.axis([-.1, np.max(lag_range)+0.1, 0, 1])
    plt.xlabel('time-lag')
    plt.ylabel('corr. of cov.')
    plt.title('time-lagged correlations')
    plt.legend(['GROUSE', 'ssidid'], loc=3)
            
    plt.subplot(1,3,3)
    plt.plot(np.arange(n), prin_angl_final[iExp,1,:], 'o', color=color_GROUSE, linewidth=3)
    plt.plot(np.arange(n)-0.05, prin_angl_final[iExp,0,:], 'o', color=color_us, linewidth=3)
    plt.plot(np.arange(n)+0.05, prin_angl_final[iExp,2,:], 'o', color='b', linewidth=3)
    plt.axis([-0.1, n+.1, 0, 1.1])
    plt.title('prin. angles betw. est. and true C')
    plt.xlabel('latent mode')
    plt.ylabel('principal angle')
    plt.box('off')
    plt.xticks(np.arange(n), ['#' + str(i+1) for i in range(n)])
    plt.legend(['GROUSE', 'ssidid', 'PCA'], loc=2)        
    
    tmp[iExp] = calc_subspace_proj_error(pars_est['C'], pars_true['C'])
    tmp_g[iExp] = calc_subspace_proj_error(pars_est_g['C'], pars_true['C'])
    tmp_pca[iExp] = calc_subspace_proj_error(pars_est_pca['C'], pars_true['C'])

    
plt.subplot(1,3,2)
m = np.mean(temp_corrs_g[idxl,:], axis=1)
s = np.std(temp_corrs_g[idxl,:], axis=1) / np.sqrt(temp_corrs_g[idxl,:].shape[1])
plt.plot(lags, m, '.-', color=color_GROUSE, linewidth=2)

m = np.mean(temp_corrs[idxl,:], axis=1)
s = np.std(temp_corrs[idxl,:], axis=1) / np.sqrt(temp_corrs[idxl,:].shape[1])
plt.plot(lags, m, '.-', color=color_us, linewidth=2)

m = np.mean(temp_corrs[idxl,:], axis=1)
s = np.std(temp_corrs[idxl,:], axis=1) / np.sqrt(temp_corrs[idxl,:].shape[1])
plt.fill_between(lags, m-s, m+s, where=m+s>=m-s, 
                 facecolor=color_us, alpha=0.3)

m = np.mean(temp_corrs_g[idxl,:], axis=1)
s = np.std( temp_corrs_g[idxl,:], axis=1) / np.sqrt(temp_corrs_g[idxl,:].shape[1])
plt.fill_between(lags, m-s, m+s, where=m+s>=m-s, 
                 facecolor=color_GROUSE, alpha=0.3)        

plt.tick_params(axis="both", which="both", top="off", right="off", labelleft="on", tickdir='out')    
plt.box('off')
#plt.axis([-.1, np.max(lag_range)+0.1, 0, 1])
plt.xlabel('time-lag')
plt.ylabel('corr. of cov.')
plt.title('time-lagged correlations (mean +/- SEM)')
plt.legend(['GROUSE', 'ssidid'], loc=3)


plt.savefig(fig_path + 'illustration_dynamics_dominated_best_setting.pdf')
        
plt.show()